In [ ]:
from keras.layers import MaxPooling2D,Conv2D,Flatten,Dense
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import optimizers
from keras import losses
from keras import metrics

In [ ]:
train_dir="/home/serkan/Belgeler/bilsem23_24/O2A2_2324/ML/kopek_duygulari/data/train"
valid_dir="/home/serkan/Belgeler/bilsem23_24/O2A2_2324/ML/kopek_duygulari/data/validation"
test_dir="/home/serkan/Belgeler/bilsem23_24/O2A2_2324/ML/kopek_duygulari/data/validation"

In [ ]:
# veri çeşitlendirme/artırma(data augmentation)  dahil edilen imaja üretici
datagen_ag = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"     
)
# veri çeşitlendirme/artırma(data augmentation)  dahil edilmemiş imaja üretici
datagen = ImageDataGenerator(
    rescale=1./255,     
)

In [ ]:
# dizinden eğitim ve doğrulama dosyalarını çekerek eğitim veri kümesi ile doğrulama veri kümsesinin oluşturulması

train_generator=datagen.flow_from_directory(
    directory=train_dir,
    target_size=(256,256),
    batch_size=4,
    class_mode="categorical"
)
valid_generator=datagen.flow_from_directory(
    directory=valid_dir,
    target_size=(256,256),
    batch_size=4,
    class_mode="categorical"
)

In [ ]:
for data_patch,label_patch in train_generator:
    print(data_patch.shape)
    print(label_patch.shape)
    break

In [ ]:
#modeli hazırlamak
model = Sequential()
model.add(Conv2D(filters=32,kernel_size=(3,3),activation="relu",input_shape=(256,256,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=64,kernel_size=(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=128,kernel_size=(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=128,kernel_size=(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(512,activation="relu"))
model.add(Dense(4,activation="softmax"))
model.summary()

In [ ]:
#modelin derlenmesi
model.compile(
    loss=losses.CategoricalCrossentropy(),
    optimizer=optimizers.RMSprop(learning_rate=1e-4),
    metrics=[metrics.Accuracy()]
    
    )

In [ ]:
#model için bir kontrol noktası callbacks oluşturmak

cp = ModelCheckpoint(
    filepath="dog_emotions_cnn.h5",
    monitor="val_loss",
    mode="min",
    save_best_only=True  
    
)

In [ ]:
history= model.fit(train_generator,validation_data=valid_generator,epochs=1)